# Replication of Dermatology Case Task 1 and 2

In [1]:
import os
import pathlib
import numpy as np
import torch
import torchvision

from PIL import Image

In [2]:
# os.path.join('a', "ISIC2018_Task1-2_Training_Input")
# root = pathlib.Path.cwd().joinpath('Data')
# root / "ISIC2018_Task1-2_Training_Input"

In [3]:
class ISICDataset(object):

    train_data_path = 'sample_train'
    ground_truth_path = 'sample_ground_Truth'
    
#     train_data_path = '"ISIC2018_Task1-2_Training_Input"'
#     ground_truth_path = 'ISIC2018_Task1_Training_GroundTruth'
    
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(root / ISICDataset.train_data_path)))
        self.masks = list(sorted(os.listdir(root / ISICDataset.ground_truth_path)))

    def __getitem__(self, idx):
        # load images ad masks
        img_path = os.path.join(self.root, ISICDataset.train_data_path, self.imgs[idx])
        mask_path = os.path.join(self.root, ISICDataset.ground_truth_path, self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path)
        # convert the PIL Image into a numpy array
        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [4]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True, progress=False)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [5]:
import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [8]:
# import utils

# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True, progress=False)
# dataset = ISICDataset('.', get_transform(train=True))
# data_loader = torch.utils.data.DataLoader(
#  dataset, batch_size=2, shuffle=True, num_workers=4,
#  collate_fn=utils.collate_fn)
# # For Training
# images,targets = next(iter(data_loader))
# images = list(image for image in images)
# targets = [{k: v for k, v in t.items()} for t in targets]
# output = model(images,targets)   # Returns losses and detections
# # For inference
# model.eval()
# x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
# predictions = model(x)           # Returns predictions

In [9]:
# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [10]:
from engine import train_one_epoch, evaluate
import utils

# our dataset has two classes only - background and person
num_classes = 2
# use our dataset and defined transformations
dataset = ISICDataset(pathlib.Path.cwd().joinpath('Data'), get_transform(train=True))

# split the dataset in train and test set
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-5])
#     dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=1, shuffle=True, num_workers=0,
    collate_fn=utils.collate_fn)


# get the model using our helper function
model = get_model_instance_segmentation(num_classes)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

# let's train it for 10 epochs
num_epochs = 10

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader, device=device)

print("That's it!")

Epoch: [0]  [0/5]  eta: 0:00:35  lr: 0.001254  loss: 9.4773 (9.4773)  loss_classifier: 0.5174 (0.5174)  loss_box_reg: 0.1094 (0.1094)  loss_mask: 8.8320 (8.8320)  loss_objectness: 0.0131 (0.0131)  loss_rpn_box_reg: 0.0053 (0.0053)  time: 7.0099  data: 0.1860  max mem: 1214
Epoch: [0]  [4/5]  eta: 0:00:02  lr: 0.005000  loss: 5.4564 (5.0913)  loss_classifier: 0.4675 (0.3350)  loss_box_reg: 0.1049 (0.0875)  loss_mask: 5.1812 (4.6218)  loss_objectness: 0.0054 (0.0267)  loss_rpn_box_reg: 0.0091 (0.0203)  time: 2.7916  data: 0.1440  max mem: 1484
Epoch: [0] Total time: 0:00:13 (2.7920 s / it)
creating index...
index created!
Test:  [0/5]  eta: 0:00:05  model_time: 0.7008 (0.7008)  evaluator_time: 0.0075 (0.0075)  time: 1.0771  data: 0.3569  max mem: 1484
Test:  [4/5]  eta: 0:00:00  model_time: 0.7008 (0.7041)  evaluator_time: 0.0040 (0.0035)  time: 0.9728  data: 0.2551  max mem: 1484
Test: Total time: 0:00:04 (0.9736 s / it)
Averaged stats: model_time: 0.7008 (0.7041)  evaluator_time: 0.004

SystemExit: 1

C:\Users\rakes\Anaconda3\envs\torch\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [11]:
%tb

SystemExit: 1

'ls' is not recognized as an internal or external command,
operable program or batch file.
